## Data Quality Evaluation

In [1]:
import pandas as pd
import numpy as np
import json

### Load the data

In [2]:
# Load receipts.json
with open('receipts.json', 'r', encoding='utf-8-sig') as file:
    data = file.read().splitlines()
    receipts = pd.json_normalize([json.loads(line) for line in data])

In [3]:
# Load users.json
data = []
with open('users.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

users = pd.DataFrame(data)

In [4]:
# Load brands.json
with open('brands.json', 'r', encoding='utf-8-sig') as file:
    data = file.read().splitlines()
    brands = pd.json_normalize([json.loads(line) for line in data])

### Data Quality Evaluation

__Summary:__

The quality of the four datasets provided shows significant room for improvement. Each table suffers from missing values to varying degrees, which impacts the integrity and reliability of the data. Also, there are issues with duplicate entries and data structure that need to be resolved for proper data utilization.

#### Receipts

##### Data Manipulation

In [5]:
# Check the loaded dataset
receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [6]:
# Rename the id column
receipts = receipts.rename(columns = {'_id.$oid': 'id'})

In [7]:
# Manipulate the date columns: Convert from UNIX timestamp to timestamp, for better visualization
receipts['createDate.$date'] = pd.to_datetime(receipts['createDate.$date'], unit='ms')
receipts['dateScanned.$date'] = pd.to_datetime(receipts['dateScanned.$date'], unit='ms')
receipts['finishedDate.$date'] = pd.to_datetime(receipts['finishedDate.$date'], unit='ms')
receipts['modifyDate.$date'] = pd.to_datetime(receipts['modifyDate.$date'], unit='ms')
receipts['pointsAwardedDate.$date'] = pd.to_datetime(receipts['pointsAwardedDate.$date'], unit='ms')
receipts['purchaseDate.$date'] = pd.to_datetime(receipts['purchaseDate.$date'], unit='ms')
# Rename the date columns
receipts = receipts.rename(columns = {'createDate.$date': 'createDate', 'dateScanned.$date': 'dateScanned', 'finishedDate.$date': 'finishedDate', 'modifyDate.$date': 'modifyDate', 'pointsAwardedDate.$date': 'pointsAwardedDate', 'purchaseDate.$date': 'purchaseDate'})

In [129]:
# Normalize the rewardsReceiptItemList column, and create a new dataframe to store it as itemList
from pandas import json_normalize
itemList = receipts.dropna(subset=['rewardsReceiptItemList'])[['id','rewardsReceiptItemList']].copy()
itemList = itemList.explode('rewardsReceiptItemList').reset_index(drop=True)
itemList = itemList.join(pd.json_normalize(itemList.pop('rewardsReceiptItemList')))
# Will do the quality detection of itemList in the following part

##### Overall Quality Detection

__Overall Findings:__
1. Contains missing values, notably in columns such as 'bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent', 'finishedDate', and 'purchaseDate'.
2. The columns 'bonusPointsEarned' and 'pointsEarned' are potentially redundant and need clarification on their purpose.
3. The 'totalSpent' column is critical, and it's concerning to see missing values here.

In [131]:
# Data overview after manipulation
receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


In [132]:
receipts.shape

(1119, 15)

In [133]:
# Check missing value
receipts.isnull().sum()

bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
id                           0
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
purchaseDate               448
dtype: int64

In [134]:
# Check data type consistency
receipts.dtypes

bonusPointsEarned                 float64
bonusPointsEarnedReason            object
pointsEarned                       object
purchasedItemCount                float64
rewardsReceiptItemList             object
rewardsReceiptStatus               object
totalSpent                         object
userId                             object
id                                 object
createDate                 datetime64[ns]
dateScanned                datetime64[ns]
finishedDate               datetime64[ns]
modifyDate                 datetime64[ns]
pointsAwardedDate          datetime64[ns]
purchaseDate               datetime64[ns]
dtype: object

##### In-Depth Exploration

In [8]:
# In-depth detection
receipts[receipts['bonusPointsEarned'].isna() & receipts['bonusPointsEarnedReason'].isna()]

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
15,NaN,NaN,NaN,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.00,5ff1e1e9b6a9d73a3a9f10f6,5ff1e1e90a7214ada1000569,2021-01-03 15:25:29.000,2021-01-03 15:25:29.000,NaT,2021-01-03 15:25:29.000,NaT,NaT
28,NaN,NaN,NaN,3.0,"[{'deleted': True, 'description': 'DELETED ITE...",REJECTED,3.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d40a7214ada1000562,2021-01-03 15:25:08.000,2021-01-03 15:25:08.000,NaT,2021-01-03 15:25:08.000,NaT,2017-10-30 00:00:00
32,NaN,NaN,500.0,9.0,"[{'barcode': '029000079236', 'description': 'P...",FINISHED,89.91,5ff36be7135e7011bcb856d3,5ff36c750a7214ada100058f,2021-01-04 19:28:53.000,2021-01-04 19:28:53.000,2021-01-04 19:28:54,2021-01-04 19:28:54.000,2021-01-04 19:28:54,2021-01-03 19:28:53
35,NaN,NaN,250.0,5.0,"[{'barcode': '044700009888', 'description': 'O...",FINISHED,49.95,5ff36a3862fde912123a4460,5ff36adb0a720f0523000590,2021-01-04 19:22:03.000,2021-01-04 19:22:03.000,2021-01-04 19:22:04,2021-01-04 19:22:04.000,2021-01-04 19:22:04,2020-12-30 06:00:00
52,NaN,NaN,350.0,1.0,"[{'barcode': '044700019917', 'description': 'O...",FINISHED,10.00,5ff370c562fde912123a5e0e,5ff3713c0a7214ada10005b6,2021-01-04 19:49:16.000,2021-01-04 19:49:16.000,2021-01-04 19:49:16,2021-01-04 19:49:16.000,2021-01-04 19:49:16,2021-01-03 19:49:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603c6adf0a720fde1000039a,2021-03-01 04:17:35.736,2021-03-01 04:17:35.736,NaT,2021-03-01 04:17:35.736,NaT,NaT
1111,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603c9e6e0a720fde100003c7,2021-03-01 07:57:34.307,2021-03-01 07:57:34.307,NaT,2021-03-01 07:57:34.307,NaT,NaT
1115,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603d0b710a720fde1000042a,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaT,2021-03-01 15:42:41.873,NaT,NaT
1116,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603cf5290a720fde10000413,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaT,2021-03-01 14:07:37.664,NaT,NaT


### ItemList

##### Data Manipulation

1. This table has a considerable number of missing values across several columns. Notably, price-related columns ('finalPrice', 'itemPrice') and details about the item ('barcode', 'description') have substantial gaps.
2. There are signs of redundancy (e.g., 'barcode' vs 'userFlaggedBarcode', 'description' vs 'userFlaggedDescription'), suggesting that the data structure needs to be reviewed and cleaned up.

In [136]:
# Data Overview
itemList.head()

,id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
itemList.shape

(6941, 35)

In [138]:
# Check missing value
itemList.isnull().sum()

id                                       0
barcode                               3851
description                            381
finalPrice                             174
itemPrice                              174
needsFetchReview                      6128
partnerItemId                            0
preventTargetGapPoints                6583
quantityPurchased                      174
userFlaggedBarcode                    6604
userFlaggedNewItem                    6618
userFlaggedPrice                      6642
userFlaggedQuantity                   6642
needsFetchReviewReason                6722
pointsNotAwardedReason                6601
pointsPayerId                         5674
rewardsGroup                          5210
rewardsProductPartnerId               4672
userFlaggedDescription                6736
originalMetaBriteBarcode              6870
originalMetaBriteDescription          6931
brandCode                             4341
competitorRewardsGroup                6666
discountedI

#### Users

##### Data Manipulation

In [8]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [55]:
# Data Manipulation
users['_id'] = users['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) else x)
users['createdDate'] = users['createdDate'].apply(lambda x: pd.to_datetime(x['$date'], unit='ms') if isinstance(x, dict) else x)
users['createdDate'] = pd.to_datetime(users['createdDate'], unit='ms')
users['lastLogin'] = users['lastLogin'].apply(lambda x: pd.to_datetime(x['$date'], unit='ms') if isinstance(x, dict) else x)
users = users.rename(columns = {'_id': 'id'})

##### Quality Detection

1. There are missing values in columns 'lastLogin', 'signUpSource', 'state'.
2. The presence of duplicate user entries necessitates a review of the user input process to avoid unnecessary data repetition.

In [64]:
# Data overview
users.head()

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI


In [58]:
users.shape

(495, 7)

In [66]:
# Check missing value
users.isnull().sum()

id               0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [60]:
# Check for duplicates
users.duplicated().sum()

283

In [65]:
# Check data type consistency
users.dtypes

id                      object
active                    bool
createdDate     datetime64[ns]
lastLogin       datetime64[ns]
role                    object
signUpSource            object
state                   object
dtype: object

#### Brands

##### Data Manipulation

1. Significant missing values exist in the columns 'category', 'categoryCode', 'topBrand', and 'brandCode'.
2. Anomalies like the missing 'topBrand' column can affect analysis involving brand performance and ranking.

In [9]:
brands.head()

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [143]:
brands = brands.rename(columns = {'_id.$oid': 'id', 'cpg.$id.$oid': 'cpg_id', 'cpg.$ref': 'cpg_ref'})

##### Quality Detection

In [145]:
# Data overview
brands.head()

,barcode,category,categoryCode,name,topBrand,id,cpg_id,cpg_ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [69]:
brands.shape

(1167, 9)

In [70]:
# Check missing value
brands.isnull().sum()

barcode           0
category        155
categoryCode    650
name              0
topBrand        612
id                0
cpg_id            0
cpg_ref           0
brandCode       234
dtype: int64

In [71]:
# Check for duplicates
brands.duplicated().sum()

0

In [73]:
# Check data type consistency
brands.dtypes

barcode         object
category        object
categoryCode    object
name            object
topBrand        object
id              object
cpg_id          object
cpg_ref         object
brandCode       object
dtype: object